In [1]:
## data collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

## load dataset

data = gutenberg.raw('shakespeare-hamlet.txt')
## save to a file

with open('hamlet.txt','w')as file:
    file.write(data)


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [2]:
##  data preprocessing

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


## load the dataset

with open('hamlet.txt','r')as file:
    text=file.read().lower()

## tokenizer the text
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words

4818

In [3]:
## create input sequences
inputsequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        inputsequences.append(n_gram_sequence)


In [4]:
## pad sequences

max_sequence_len = max([len(x) for x in inputsequences])
max_sequence_len

14

In [5]:
inputsequences=np.array(pad_sequences(inputsequences,maxlen=max_sequence_len,padding='pre'))
inputsequences


array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [6]:
# create predictors and label

import tensorflow as tf
X,y = inputsequences[:,:-1],inputsequences[:,-1]


In [7]:
y = tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
## split the data int training and testing

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)


In [9]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

In [10]:
## train our LSTM rnn

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout


## define the model
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))

## compile the model




/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [14]:
model.build(input_shape=(None, max_sequence_len))
model.summary()
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 14, 100)             │         481,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 14, 150)             │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 14, 150)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100)                 │         100,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 4818)                │         486,618 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,658,256 (13.96 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,438,838 (9.30 MB)

In [15]:
## train the model
history = model.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),verbose=1)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 40s 57ms/step - accuracy: 0.0338 - loss: 6.6675 - val_accuracy: 0.0328 - val_loss: 6.9984
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 36s 56ms/step - accuracy: 0.0389 - loss: 6.3797 - val_accuracy: 0.0478 - val_loss: 7.0492
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 41s 56ms/step - accuracy: 0.0445 - loss: 6.2152 - val_accuracy: 0.0546 - val_loss: 7.0678
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 41s 56ms/step - accuracy: 0.0604 - loss: 6.0231 - val_accuracy: 0.0577 - val_loss: 7.0658
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 34s 53ms/step - accuracy: 0.0648 - loss: 5.8390 - val_accuracy: 0.0622 - val_loss: 7.1358
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 36s 56ms/step - accuracy: 0.0731 - loss: 5.7134 - val_accuracy: 0.0624 - val_loss: 7.2214
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 41s 55ms/step - accuracy: 0.0795 - loss: 5.5768 - val_accuracy: 0.0670 - val_loss: 7.2815
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 34s 53ms/step - accuracy: 0.0888 - loss: 5.4759 - 

In [16]:
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]
    token_list = pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted = model.predict(token_list,verbose=0)
    predicted_word_index = np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [37]:
input_text = "They are so fortified against your"
print(f'Input text:{input_text}')
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Next word prediction: {next_word}")

Input text:They are so fortified against your
Next word prediction: browes


In [32]:
## model save
model.save("next_word_lstm.h5")

## save the tokenizer
import pickle
with open('tokenzer.pickle','wb')as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)